In [64]:
import os
import requests
import json
import pandas as pd
#import yfinance as yf
#import pendulum
import matplotlib.pyplot as plt
from MCForecastTools import MCSimulation

from dotenv import load_dotenv
import alpaca_trade_api as tradeapi


%matplotlib inline

In [65]:
load_dotenv('keys.env')

True

In [66]:
import numpy as np
#from pandas_datareader import data as wb
import seaborn as sns
from scipy.stats import norm
import datetime as dt

import pytz

In [67]:
#start_date=dt.datetime(2018,1,1)
#end_date=dt.datetime(2020,1,1)
#print(start)

In [68]:
#ticker = 'TSLA'
#data = pd.DataFrame(['TSLA'])
#wb.DataReader("TSLA", 'yahoo', start=start_date, end=end_date)["Close"]
#Plot
#data.plt(figsize=(15,6))

In [69]:
#price_history = yf.Ticker('TSLA').history(period='2y', # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
                                   #interval='1mo', # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
                                   #actions=False)
#time_series = list(price_history['Close'])
#dt_list = [pendulum.parse(str(dt)).float_timestamp for dt in list(price_history.index)]

#plt.plot(time_series)
#display(dt_list)
#display(time_series)


In [70]:
#political_portfolio_dict_stock_list = [{'Mark_Warner': ['SPY', 'AKREX', 'TROW', 'COHOX', 'VTI']}, 
 #{'Greg_Gianforte': ['QQQ', 'SPY', 'VTI', 'FICO', 'WK', 'RHT']},
 #{'Nancy_Pelosi': ['AAPL', 'V', 'DIS', 'CRM', 'META', 'CMCSA']}]
                 
#Mark Warner: SPDR s&p 500 etf, Akre focus fund, t. rowe price group, coho relative value equity fund, vanguard 

#Greg Gianforte: invesco qqq, SPDr s&p 500, vanguard, FICO, Workiva, red hat inc

#Nancy Pelosi: apple, visa, disney, salesforce, facebook, comcast

In [71]:
warner_weights=[0.24, 0.23, 0.22, 0.22, 0.09]
warner_investment_capital=116000000.00

#Warner range of investments from $80m to $348m. Maximum value of assets assumed in calculations.
#investment capital reduced by 2/3 to allow for easier analysis and accomodate top 1/3 of stock 
#investment portfolio. Net worth assumed as investment capital. AMG GW&K Municipal bond fund removed from 
#top 1/3 of stock assets to simplify calculations.

gianforte_weights=[0.26, 0.22, 0.35, 0.08, 0.05, 0.04]
gianforte_investment_capital = 94500000.00

#weights of stocks making up 1/2 of Gianforte investment portfolio. Different Vanguard funds summed. 
#investment capital reduced by 1/2 to allow for easier analysis and accomodate top
#1/2 of stock investment portfolio. Net worth assumed as investment capital. 

pelosi_weights=[0.40, 0.40, 0.08, 0.08, 0.02, 0.02]
pelosi_investment_capital= 64000000.00
                
#Pelosi range of investments from $54m to $256m. Max value investments/assets assumed in calculations. 
#investment capital reduced by 3/4 to allow for easier analysis and accomodate top
#1/4 of stock investment portfolio. Net worth assumed as investment capital.

In [72]:
warner_tickers= ['SPY', 'AKREX', 'TROW', 'COHOX', 'VTI']
gianforte_tickers=['QQQ', 'SPY', 'VTI', 'FICO', 'WK', 'CRM']
pelosi_tickers=['AAPL', 'V', 'DIS', 'CRM', 'META', 'CMCSA']

#timeframe='1Day'

#start_date = pd.Timestamp("2020-08-07", tz="America/New_York").isoformat()
#end_date = pd.Timestamp("2020-08-07", tz="America/New_York").isoformat()

In [73]:
#warner_stock_info=yf.Tickers(warner_tickers)
#warner_df=warner_stock_info.history(period='5y')
#warner_df=pd.DataFrame(warner_df)
#warner_df.columns = pd.MultiIndex.from_product([warner_df.columns, ['Close']])
#warner_close_data=warner_df['Close']
#warner_close_data=warner_close_data.reindex(columns=warner_tickers)
#display(warner_close_data)

In [75]:
start_date = pd.Timestamp("2017-05-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2020-05-01", tz="America/New_York").isoformat()
timeframe="1Day"
load_dotenv('keys.env')

# Set Alpaca API key and secret key
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)
print(warner_tickers)
warner_prices_df = alpaca.get_bars(warner_tickers,timeframe,start_date,end_date).df
#warner_stock_info=yf.Tickers(warner_tickers)
#warner_df=warner_stock_info.history(period='5y')
#warner_df=pd.DataFrame(warner_df)
#warner_close_data=warner_df['Close']
#warner_close_data=warner_close_data.reindex(columns=warner_tickers)  
spy_df = warner_prices_df[warner_prices_df['symbol']=='SPY'].drop('symbol',axis=1)
akrex_df = warner_prices_df[warner_prices_df['symbol']=='SPY'].drop('symbol',axis=1)
trow_df = warner_prices_df[warner_prices_df['symbol']=='SPY'].drop('symbol',axis=1)
cohox_df = warner_prices_df[warner_prices_df['symbol']=='SPY'].drop('symbol',axis=1)
vti_df = warner_prices_df[warner_prices_df['symbol']=='SPY'].drop('symbol',axis=1)
warner_prices_df = pd.concat([spy_df, akrex_df, trow_df, cohox_df, vti_df],axis=1,keys=['SPY', 'AKREX', 'TROW', 'COHOX', 'VTI'])
#display(warner_prices_df)
#warner_returns=warner_close_data.pct_change()
#mean_warner_returns=warner_returns.mean()
#std_warner_returns=warner_returns.std()
warner_MC_30yr = MCSimulation(
    portfolio_data = warner_prices_df,
    weights = warner_weights,
    num_simulation = 500,
    num_trading_days = 252*30
)

# Review the simulation input data
#MC_30yr.portfolio_data.head()

['SPY', 'AKREX', 'TROW', 'COHOX', 'VTI']
[*********************100%***********************]  5 of 5 completed


KeyError: 'Close'

In [26]:
warner_MC_30yr.calc_cumulative_return()

NameError: name 'warner_MC_30yr' is not defined

In [ ]:
warner_MC_sim_line_plot = warner_MC_30yr.plot_simulation()

In [ ]:
warner_MC_summary_stats = warner_MC_30yr.summarize_cumulative_return()
print(warner_MC_summary_stats)

In [ ]:
#warner_returns=warner_close_data.pct_change()
#warner_returns.head()
#mean_warner_returns=warner_returns.mean()
#std_warner_returns=warner_returns.std()

#display(warner_returns.dropna())

In [ ]:
#warner_portfolio_returns=warner_returns.dot(warner_weights)
#warner_portfolio_returns.head()

In [ ]:
#warner_variance=warner_returns.cov()*252
#warner_variance

In [ ]:
#warner_portfolio_variance=np.transpose(warner_weights)@warner_variance@warner_weights
#warner_portfolio_volitility=np.sqrt(warner_portfolio_variance)
#display(f"warner_portfolio_variance is", warner_portfolio_variance)
#display(f"warner_portfolio_volitility is", warner_portfolio_volitility)

In [ ]:
#gianforte_stock_info=yf.Tickers(gianforte_tickers)
#gianforte_df=gianforte_stock_info.history(period='5y')
#gianforte_df=pd.DataFrame(gianforte_df)
#gianforte_close_data=gianforte_df['Close']
#gianforte_close_data=gianforte_close_data.reindex(columns= gianforte_tickers)
#display(gianforte_close_data)

In [76]:
print(gianforte_tickers)
gianforte_prices_df = alpaca.get_bars(gianforte_tickers,timeframe,start_date,end_date).df

['QQQ', 'SPY', 'VTI', 'FICO', 'WK', 'CRM']


In [77]:
print(gianforte_tickers)
gianforte_prices_df = alpaca.get_bars(gianforte_tickers,timeframe,start_date,end_date).df
#warner_stock_info=yf.Tickers(warner_tickers)
#warner_df=warner_stock_info.history(period='5y')
#warner_df=pd.DataFrame(warner_df)
#warner_close_data=warner_df['Close']
#warner_close_data=warner_close_data.reindex(columns=warner_tickers)  
qqq_df = gianforte_prices_df[gianforte_prices_df['symbol']=='QQQ'].drop('symbol',axis=1)
spy_df = gianforte_prices_df[gianforte_prices_df['symbol']=='QQQ'].drop('symbol',axis=1)
vti_df = gianforte_prices_df[gianforte_prices_df['symbol']=='QQQ'].drop('symbol',axis=1)
fico_df =gianforte_prices_df[gianforte_prices_df['symbol']=='QQQ'].drop('symbol',axis=1)
wk_df = gianforte_prices_df[gianforte_prices_df['symbol']=='QQQ'].drop('symbol',axis=1)
crm_df = gianforte_prices_df[gianforte_prices_df['symbol']=='QQQ'].drop('symbol',axis=1)
gianforte_prices_df = pd.concat([qqq_df,spy_df,vti_df,fico_df,wk_df,crm_df],axis=1,keys=['QQQ', 'SPY', 'VTI', 'FICO', 'WK', 'CRM'])


#warner_returns=warner_close_data.pct_change()
#mean_warner_returns=warner_returns.mean()
#std_warner_returns=warner_returns.std()
gianforte_MC_30yr = MCSimulation(
    portfolio_data = gianforte_prices_df,
    weights = gianforte_weights,
    num_simulation = 500,
    num_trading_days = 252*30
)

# Review the simulation input data
gianforte_MC_30yr.portfolio_data.head()

['QQQ', 'SPY', 'VTI', 'FICO', 'WK', 'CRM']


KeyError: 'Close'

In [ ]:
gianforte_MC_30yr.calc_cumulative_return()

In [ ]:
gianforte_MC_summary_stats = gianforte_MC_30yr.summarize_cumulative_return()
print(gianforte_MC_summary_stats)

In [ ]:
gianforte_MC_30yr.calc_cumulative_return()

In [ ]:
gianforte_MC_sim_line_plot = gianfote_MC_30yr.plot_simulation()

In [ ]:
gianforte_returns=gianforte_close_data.pct_change()
mean_gianforte_returns=gianforte_returns.mean()
std_gianforte_returns=gianforte_returns.std()

In [ ]:
#pelosi_stock_info=yf.Tickers(pelosi_tickers)
#pelosi_df=pelosi_stock_info.history(period='5y')
#pelosi_df=pd.DataFrame(pelosi_df)
#pelosi_close_data=pelosi_df['Close']
#pelosi_close_data=pelosi_close_data.reindex(columns= pelosi_tickers)
#display(pelosi_close_data)

In [ ]:
print(pelosi_tickers)
pelosi_prices_df = alpaca.get_bars(pelosi_tickers,timeframe,start_date,end_date).df
#warner_stock_info=yf.Tickers(warner_tickers)
#warner_df=warner_stock_info.history(period='5y')
#warner_df=pd.DataFrame(warner_df)
#warner_close_data=warner_df['Close']
#warner_close_data=warner_close_data.reindex(columns=warner_tickers)  
aapl_df = pelosi_prices_df[pelosi_prices_df['symbol']=='AAPL'].drop('symbol',axis=1)
v_df = pelosi_prices_df[pelosi_prices_df['symbol']=='AAPL'].drop('symbol',axis=1)
dis_df = pelosi_prices_df[pelosi_prices_df['symbol']=='AAPL'].drop('symbol',axis=1)
crm_df =pelosi_prices_df[pelosi_prices_df['symbol']=='AAPL'].drop('symbol',axis=1)
meta_df = pelosi_prices_df[pelosi_prices_df['symbol']=='AAPL'].drop('symbol',axis=1)
cmcsa_df = pelosi_prices_df[pelosi_prices_df['symbol']=='AAPL'].drop('symbol',axis=1)
pelosi_prices_df = pd.concat([aapl_df,v_df,dis_df,crm_df,meta_df,cmcsa_df],axis=1,keys=['QQQ', 'SPY', 'VTI', 'FICO', 'WK', 'CRM'])


#warner_returns=warner_close_data.pct_change()
#mean_warner_returns=warner_returns.mean()
#std_warner_returns=warner_returns.std()
pelosi_MC_30yr = MCSimulation(
    portfolio_data = pelosi_prices_df,
    weights = pelosi_weights,
    num_simulation = 500,
    num_trading_days = 252*30
)

# Review the simulation input data
pelosi_MC_30yr.portfolio_data.head()

In [ ]:
pelosi_MC_sim_line_plot = pelosi_MC_30yr.plot_simulation()

In [ ]:
pelosi_MC_summary_stats = pelosi_MC_30yr.summarize_cumulative_return()
print(pelosi_MC_summary_stats)

In [ ]:
pelosi_returns=pelosi_close_data.pct_change()
mean_pelosi_returns=pelosi_returns.mean()
std_pelosi_returns=pelosi_returns.std()

In [ ]:
spy_shares_warner= 110476
akrex_shares_warner= 762286
trow_shares_warner= 220000
cohox_shares_warner= 1772222
vti_shares_warner= 74359

qqq_shares_gianforte= 149271
spy_shares_gianforte= 82500
vti_shares_gianforte= 235577
fico_shares_gianforte= 39171
wk_shares_gianforte= 181730
rht_shares_gianforte= 25116

aapl_shares_pelosi=548180
visa_shares_pelosi=193939
dis_shares_pelosi=46694
crm_shares_pelosi=37372
meta_shares_pelosi=7442
cmcsa_shares_pelosi=35774

#total shares calculated based on 2018 median share prices for each stock in portfolio b/c most recent portfolio data
#from 2018. should 

In [ ]:
stock_investment= pelosi_investment_capital * 0.02
#spy 2018 median share price =252
#akrex 2018 median share price = 35
#trow 2018 median share price = 116
#cohox 2018 median share price = 14.4
#vti 2018 median share price = 140.4

#qqq 2018 med share price = 164.6
#fico 2018 med share price = 193
#wk 2018 med share price = 26
#rht 2018 med share price = 150.5

#aapl 2018 med share price = 46.7
#visa 2018 med share price = 132
#disney 2018 med share price = 109.65
#crm 2018 med share price = 137
#meta 2018 med share price = 172
#cmcsa 2018 med share price = 35.78

In [ ]:
stock_investment/35.78